#### This notebook measures the error of an .swc output and a marker file with manually labeled corners.

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
import os

In [3]:
import networkx as nx

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
data_path = "DATA_example_Holco_praire_Scan29"

In [7]:
SWCFile = os.path.join("..",data_path,"processed","surfaces","thresholded_image_SIGEN.swc")

In [8]:
### read .swc file
SWCMarkers = pd.read_csv(SWCFile, skiprows = 3,sep = " ",header = None)
SWCMarkers.columns = ["n","type","x","y","z","radius","parent"]
SWCMarkers.head()

,n,type,x,y,z,radius,parent
0,1,6,39.0,210.000,123.000,0.000,-1
1,2,3,39.0,209.500,122.000,0.500,1
2,3,3,39.0,209.000,121.000,0.000,2
3,4,3,40.0,208.000,120.000,0.000,3
4,5,5,40.0,209.333,119.167,2.339,4


In [9]:
# Extract ones which are not parents 

In [10]:
df = SWCMarkers[['n','parent','type','x','y','z']]

In [11]:
df_noroots = df[df.parent!=-1]
#df_noroots['pos'] = list(zip(df_noroots['x'],df_noroots['y'],df_noroots['z']))

In [12]:
df_noroots.head()

,n,parent,type,x,y,z
1,2,1,3,39.0,209.500,122.000
2,3,2,3,39.0,209.000,121.000
3,4,3,3,40.0,208.000,120.000
4,5,4,5,40.0,209.333,119.167
5,7460,5,6,40.0,210.000,120.000


In [13]:
G = nx.from_pandas_dataframe(df_noroots, 'n', 'parent')

In [14]:
#pos_df = nx.get_edge_attributes(G,'pos')
pos_df = SWCMarkers[['n','x','y','z']]

In [15]:
pos_df.head()

,n,x,y,z
0,1,39.0,210.000,123.000
1,2,39.0,209.500,122.000
2,3,39.0,209.000,121.000
3,4,40.0,208.000,120.000
4,5,40.0,209.333,119.167


In [16]:
pos_dict = {}
for n, pos in zip(pos_df.n,list(zip(pos_df.x,pos_df.y,pos_df.z))):
    pos_dict[n] = pos

In [17]:
# networkx does not work with 3d coordinates
# plt.figure()
# nx.draw(G,pos_dict,node_size = 1)

### Plotting 3D Graphs with Plotly

In [18]:
import plotly

In [38]:
plotly.tools.set_credentials_file(username='vms', api_key='')

In [20]:
# creating function to plot networkx graphs
import plotly.plotly as py
from plotly.graph_objs import *

import networkx as nx

def plot_graph(G):

    pos_dict_small = []

    #for item in G.nodes():
    #    pos_dict_small[item] = pos_dict[item]
    #nx.set_node_attributes(G,'pos',pos_dict)
    pos=nx.get_node_attributes(G,'pos')

    dmin=1
    ncenter=0
    for n in pos:
        x,y,z=pos[n]
        d=(x-0.5)**2+(y-0.5)**2+(z-0.5)**2
        if d<dmin:
            ncenter=n
            dmin=d

# p=nx.single_source_shortest_path_length(G,ncenter)


    edge_trace = Scatter3d(
        x=[],
        y=[],
        z=[],
        line=Line(width=0.5,color='#888'),
        hoverinfo='none',
        mode='lines')

    for edge in G.edges():
        x0, y0, z0 = G.node[edge[0]]['pos']
        x1, y1, z1 = G.node[edge[1]]['pos']
        edge_trace['x'] += [x0, x1, None]
        edge_trace['y'] += [y0, y1, None]
        edge_trace['z'] += [z0, z1, None]

    node_trace = Scatter3d(
        x=[],
        y=[],
        z=[],
        text=[],
        mode='markers',
        hoverinfo='text',
        marker=Marker(
            showscale=True,
            # colorscale options
            # 'Greys' | 'Greens' | 'Bluered' | 'Hot' | 'Picnic' | 'Portland' |
            # Jet' | 'RdBu' | 'Blackbody' | 'Earth' | 'Electric' | 'YIOrRd' | 'YIGnBu'
            colorscale='YIGnBu',
            reversescale=True,
            color=[],
            size=1,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line=dict(width=2)))

    for node in G.nodes():
        x, y, z = G.node[node]['pos']
        node_trace['x'].append(x)
        node_trace['y'].append(y)
        node_trace['z'].append(z)

    for node, adjacencies in enumerate(G.adjacency_list()):
        node_trace['marker']['color'].append(len(adjacencies))
        node_info = '# of connections: '+str(len(adjacencies))
        node_trace['text'].append(node_info)

    
    fig = Figure(data=Data([edge_trace, node_trace]),
                 layout=Layout(
                    title='<br>Network graph made with Python',
                    titlefont=dict(size=16),
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    annotations=[ dict(
                        text="Python code: <a href='https://plot.ly/ipython-notebooks/network-graphs/'> https://plot.ly/ipython-notebooks/network-graphs/</a>",
                        showarrow=False,
                        xref="paper", yref="paper",
                        x=0.005, y=-0.002 ) ],
                    xaxis=XAxis(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=YAxis(showgrid=False, zeroline=False, showticklabels=False)))
    return(fig)

    py.iplot(fig, filename='networkx')

In [21]:
# adding the positions as attributes
nx.set_node_attributes(G,'pos',pos_dict)

In [22]:
fig = plot_graph(G)

In [23]:
py.iplot(fig,filename='raw')

### Node Degree

In [24]:
nx.set_node_attributes(G,'degree',G.degree(G.nodes()))

In [25]:
# nx.get_node_attributes(G,'degree')

In [26]:
G.remove_nodes_from([n for n,d in G.degree_iter() if d>4])

In [27]:
#G.remove_nodes_from([n for n,d in G.degree_iter() if d<2])

In [28]:
fig = plot_graph(G)
py.iplot(fig,filename='networkx')

Note: removing edges with degree less than 2 creates a lot of disconnections in the pattern.

### Drop edges with large edge length

In [29]:
#Compute the node-to-node distances
import math
lengths={}
for edge in G.edges():
   startnode=edge[0] 
   endnode=edge[1]
   lengths[edge]=round(math.sqrt(((pos_dict[endnode][1]-pos_dict[startnode][1])**2)+
                                      ((pos_dict[endnode][0]-pos_dict[startnode][0])**2) + 
                                       ((pos_dict[endnode][2]-pos_dict[startnode][2])**2)),2) #The distance

In [30]:
nx.set_edge_attributes(G,'dist',lengths)

In [31]:
G.remove_edges_from([(a,b) for a,b,v in G.edges(data=True) if v['dist']>5])

In [32]:
fig = plot_graph(G)
py.iplot(fig,filename='networkx')

### Closing cycles
* nodes of close distances
* check if there is path between them
* if so add edge

In [33]:
from sklearn.metrics.pairwise import pairwise_distances as pdist

In [34]:
D = pdist(pd.DataFrame(nx.get_node_attributes(G,'pos')).transpose(),pd.DataFrame(nx.get_node_attributes(G,'pos')).transpose())

In [36]:
labels = pd.DataFrame(nx.get_node_attributes(G,'pos')).columns
labels

Float64Index([   1.0,    2.0,    3.0,    4.0,    5.0,    6.0,    7.0,    8.0,
                 9.0,   10.0,
              ...
              8235.0, 8236.0, 8237.0, 8238.0, 8239.0, 8240.0, 8241.0, 8242.0,
              8243.0, 8244.0],
             dtype='float64', length=8082)

In [37]:
indeces = np.argwhere(D<5)
for idx in indeces:
    if not G.has_edge(int(labels[idx[0]]),int(labels[idx[1]])):
        try:
            spl = nx.shortest_path_length(G, labels[idx[0]],labels[idx[1]], weight='dist')
            #print(spl)
            #print(D[idx[0],idx[1]])
            print(D[idx[0],idx[1]]/spl)
            if D[idx[0],idx[1]]/spl<0.3:
                G.add_edge(labels[idx[0]],labels[idx[1]], dist = D[idx[0],idx[1]])
        except:
            G.add_edge(labels[idx[0]],labels[idx[1]], dist = D[idx[0],idx[1]])
            pass

/Users/valentina/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning:

invalid value encountered in double_scalars

/Users/valentina/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning:

invalid value encountered in double_scalars



nan
0.998244632812
0.942482968961
0.725101239324
0.478408118028
0.698213835751
nan
0.944765755638
0.680757839508
0.607859609024
0.749474012747
0.41735662067
0.758055947784
0.582647954619
0.998244632812
nan
0.640733767838
0.56350261849
0.607859609024
0.740429037459
0.710476851931
0.759531703189
0.396350299215
0.642931604714
0.542592506658
0.942482968961
0.944765755638
nan
0.455410993381
0.530938991495
0.59390915737
0.268648895955
0.495565580002
0.894564826946
0.669974080544
0.923495538886
0.757575757576
0.725101239324
0.680757839508
0.640733767838
nan
0.978688071065
0.662795621096
0.595387764519
0.867821661913
0.924173713192
0.379882914664
0.376148735352
0.607859609024
0.56350261849
0.455410993381
nan
0.853461060114
0.846035431803
0.657126281763
0.418889613679
0.386172298434
0.664416700927
0.82945076972
0.795288877527
0.462249614792
0.934944586308
0.607859609024
0.530938991495
0.978688071065
nan
0.830479805786
0.696788442563
0.481689747132
0.417497766385
0.658710231576
0.425944776182
0.

0.820038330016
0.266688313067
0.479740192863
0.579543276843
0.592634549358
0.619785470726
0.333138064022
0.848822832876
0.847238249813
nan
0.845525556526
0.821969784033
0.941034780342
0.937881091618
0.861639145361
0.225900068011
nan
0.984396770507
0.698137616811
0.69929729989
0.667960945919
0.812625785964
0.773697790688
0.226985042528
0.586797168377
0.845525556526
nan
0.852794445179
0.702981830528
0.71839585897
0.77929567189
0.673838036808
0.60142089535
0.330119139802
0.821969784033
0.984396770507
nan
0.765968297354
0.695999628888
0.460584666748
0.448268753151
0.346786377142
0.146687798179
0.852794445179
0.765968297354
inf
0.853363662044
0.965113220625
0.893962513235
0.73926069156
0.0663756126136
0.12398835696
0.233201160156
nan
0.920787432478
0.440959832046
0.352192556833
0.266114436034
0.36363802572
0.249918411933
0.735213406621
0.263925039038
0.853363662044
nan
0.854540094818
0.225123326901
0.621834925777
0.500376896313
0.333333333333
0.889039145668
0.87948259207
0.747358170949
0.69

0.254503302853
0.886479842205
0.881983690101
nan
0.310070282399
0.28981316563
0.496907614646
0.194766727141
0.781573283946
0.731765319328
0.63418241824
0.379779543049
nan
0.209089216115
0.336963807685
0.351371426364
0.18340296669
0.636868982041
0.853779902675
0.828172043324
nan
0.90773055657
0.888053793158
0.870400038459
nan
0.803787571363
0.206363287924
0.602525294395
0.38951156741
0.998422666288
nan
0.936325418571
0.821339800234
0.674265463919
0.358563446182
0.199258284465
0.848662264782
0.803787571363
nan
0.826873107438
0.775623122863
0.74561063045
0.876482397827
0.902352014588
0.710330943102
0.970387395589
0.936325418571
nan
0.991271188579
0.946371909829
0.88975742431
0.214361685804
0.985603764155
0.821339800234
0.826873107438
nan
0.963501825739
0.859471421304
0.840971282045
0.908693831641
0.775623122863
0.991271188579
nan
0.826436689748
0.411666590893
0.589313639839
0.74561063045
0.946371909829
0.963501825739
nan
0.859471421304
0.826436689748
nan
0.990707070198
0.411469407157
0.33

0.482883461088
0.224273430494
0.478803225684
0.407556422509
0.586503455774
0.470030437008
0.895593711639
nan
0.979035214987
0.779239613696
0.762034017129
0.867463754782
0.549719685598
0.534279157167
0.963527661062
0.820743406967
nan
0.694875089524
0.705587334035
0.737725327945
0.931382852857
0.569509459087
0.979035214987
nan
0.98521268989
0.98057580181
0.917206507369
0.939828298645
0.779239613696
0.694875089524
nan
0.996866546312
0.947296126234
0.614756778537
0.952983709228
0.859455613606
0.762034017129
0.705587334035
0.98521268989
nan
0.952770099323
0.550767210598
0.56642239596
0.909940233696
0.737725327945
0.98057580181
0.996866546312
nan
0.46997942989
0.521659796739
0.997608394188
0.881316287853
0.947296126234
0.952770099323
nan
0.986944694179
0.9695218648
0.949867763877
0.971770314044
0.614756778537
0.550767210598
0.46997942989
nan
0.994054124624
0.822392913275
0.750400631754
0.914904419837
0.986185147896
0.56642239596
0.521659796739
0.986944694179
nan
0.772794296378
0.709500023496

...

0.986658034981
nan
0.986496048773
0.976043391215
0.980515650415
0.973425631736
0.98643201688
0.998873910688
nan
0.976621234622
0.980502395763
0.97993620038
0.983070435227
0.990045321836
0.986496048773
nan
0.985788487879
0.97941793495
0.982437420368
0.980190732153
0.976043391215
0.976621234622
nan
0.990542685097
0.991544722097
0.983273744076
0.980515650415
0.980502395763
0.985788487879
inf
0.998752684706
0.980241456239
0.93482575352
0.97993620038
0.97941793495
0.990542685097
nan
0.982110729356
0.924622787653
0.926223181915
0.982437420368
0.991544722097
0.998752684706
nan
0.897364711413
0.914252273784
0.747192941529
0.983273744076
0.980241456239
0.982110729356
nan
0.991161778284
0.924707449528
0.687193997557
0.740651068045
0.93482575352
0.924622787653
0.897364711413
nan
0.924305324935
0.925214179686
0.86842387911
0.769338741236
0.792196121192
0.926223181915
0.914252273784
0.991161778284
nan
0.984161456902
0.923679941844
0.90188647358
0.917209778639
0.924707449528
0.924305324935
nan
0.942

0.941141689981
0.944413604341
0.939055580948
0.960080944982
0.952845868666
0.947018010518
nan
0.985051972467
0.949428666082
0.954509291233
0.957965544475
0.949807903188
0.941141689981
nan
0.9375
0.95099162859
0.960080944982
0.953694597233
0.944413604341
0.985051972467
nan
0.967299532202
0.966946003019
0.970404692825
0.939055580948
0.949428666082
0.9375
nan
0.998392291216
0.99824279872
0.990724267981
0.954509291233
0.95099162859
0.967299532202
nan
0.998865569687
0.988211768803
0.983577607207
0.960080944982
0.966946003019
0.998392291216
nan
0.986660624911
0.984161658859
0.973113717991
0.970404692825
0.99824279872
0.998865569687
nan
0.992315899416
0.967923230285
0.96698653687
0.990724267981
0.988211768803
0.986660624911
nan
0.965149881065
0.971680513342
0.970020553346
0.983577607207
0.984161658859
0.992315899416
nan
0.985051972467
0.976762986629
0.963852865161
0.973113717991
0.967923230285
0.965149881065
nan
0.976056869423
0.968415296944
0.974630946585
0.96698653687
0.971680513342
0.98505

0.999444213691
1.00062280357
0.794239799625
0.879147991028
0.958583802196
0.949924997924
0.986505191175
nan
1.00062280357
0.869377382208
0.725087560611
0.808080808081
0.759493670886
0.728463891452
nan
0.706214689266
0.735364635281
0.996205315804
0.649000492834
0.952927387076
0.988639199482
0.999444213691
0.706214689266
nan
0.852678848427
0.75992796339
0.65038514204
0.752635717239
1.00062280357
1.00062280357
0.735364635281
nan
0.947207727431
0.663327186854
0.856586358591
0.869377382208
0.852678848427
nan
0.551241993128
0.509142986212
0.439593724209
0.400986580393
0.725087560611
0.75992796339
0.947207727431
nan
0.707500384144
0.982770406791
0.536155457116
0.629287423311
0.70239752082
0.794239799625
0.996205315804
0.65038514204
0.663327186854
0.551241993128
0.707500384144
nan
nan
0.890723542831
0.862572075469
0.819780982997
inf
0.850514398665
0.813789851445
0.890723542831
nan
0.957850035825
0.862572075469
0.850514398665
nan
0.819780982997
0.813789851445
0.957850035825
nan
nan
0.8740719245

0.612574113277
0.538214110635
1.0
nan
0.553455635637
0.452233597192
0.606091526731
0.414578098794
0.323041910998
0.974996043044
nan
0.535090008317
0.414963200467
0.471404520791
0.38643699843
0.974996043044
nan
0.758055947784
0.642931604714
0.545648509726
0.723853742727
0.376805965675
0.866244775059
0.612574113277
0.397944843452
0.302078371031
0.5085710234
0.490520177375
nan
0.698213835751
0.582647954619
0.542592506658
0.519486193257
0.701308155656
0.966450192656
0.352867502482
0.720759220056
0.50973889992
0.441784514024
0.624135439581
nan
0.55943369395
0.529150262213
0.538214110635
0.612574113277
0.796829299038
0.664885531446
0.821854415127
0.866244775059
0.720759220056
0.612574113277
0.585100424599
nan
0.619769775572
0.707351921468
0.724807465442
0.796829299038
0.821854415127
0.866244775059
0.612574113277
0.486832980505
0.441518440112
nan
0.698477705519
0.379795897113
0.634464704271
0.689107081986
0.817322060303
0.891520879557
0.86969384567
0.64823151951
nan
0.445215028797
0.626403042

/Users/valentina/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning:

divide by zero encountered in double_scalars

/Users/valentina/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning:

divide by zero encountered in double_scalars



### Connected components

In [39]:
fig = plot_graph(G)
py.iplot(fig,filename='closed_cycles')

In [40]:
cc_graphs = list(nx.connected_component_subgraphs(G))

In [41]:
# find graphs with most components

In [42]:
l = []
for graph in cc_graphs:
    l.append(len(graph.nodes()))

In [43]:
index = sorted(range(len(l)), key=lambda k: l[k])

In [44]:
cc_graphs_sorted = [cc_graphs[i] for i in index][::-1]

In [45]:
len(cc_graphs_sorted[3])

131

In [46]:
# nx.cycle_basis(G)


[[2, 5, 6, 1], [2, 3, 7, 5], [2, 4, 8, 3], [9, 4, 8, 3, 7, 5, 6, 1]]

In [49]:
max(list(cc_graphs[0].degree().values()))

7

In [50]:
# plot first n connected components
U = nx.Graph()

for graph in cc_graphs_sorted[:40]: 
    
    if len(nx.cycle_basis(graph))>0:
        no_cycle = False
    else:
        no_cycle = True
    
    if not no_cycle:
        U.add_edges_from(graph.edges(data=True))
        U.add_nodes_from(graph.nodes(data=True)) 
fig = plot_graph(U)
py.iplot(fig,filename='networkx')

In [51]:
# plot first n connected components
U = nx.Graph()

for graph in cc_graphs_sorted[:40]: 
    max_d = max(list(graph.degree().values()))
    ave_d = sum(list(graph.degree().values()))/len(graph.nodes())
    #ave_d = sum(list(graph.degree().values()==2))
    ave_d = [item for item in graph.degree().values() if item>2]
    if len(ave_d)/len(graph.nodes())>0.05:
        graph.degree()
        U.add_edges_from(graph.edges(data=True))
        U.add_nodes_from(graph.nodes(data=True)) 
fig = plot_graph(U)
py.iplot(fig,filename='no_cycles')

In [52]:
# remove roots with big radius
small_roots = roots[roots.radius<2]
small_roots.shape
# seems none of the roots has a big radius???

NameError: name 'roots' is not defined

In [ ]:
SWCMarkers.radius.hist(bins = 20)

In [ ]:
plt.figure()
SWCMarkers.type.hist()
plt.show()

In [ ]:
print("number of dendrites: ",sum(SWCMarkers.type==3))
print("number of fork points: ",sum(SWCMarkers.type==5))
print("number of end_points: ",sum(SWCMarkers.type==6))

Remove nodes with large radius:

In [ ]:
pruned = SWCMarkers[(SWCMarkers.type==6)&(SWCMarkers.radius<2)]

In [ ]:
len(SWCMarkers)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pruned.x,pruned.y,pruned.z)
plt.show()